In [45]:
import pandas as pd
import re
import operator
import string
import nltk 
from nltk.tag import pos_tag
pd.set_option('display.width', 10000)

In [46]:
stopword = nltk.corpus.stopwords.words('english')
stopword.extend(['best', 'picture', 'musical', 'Best', 'Picture', 'Musical', 'Comedy', 'comedy', 'RT'])

In [47]:
gg = pd.read_json('gg2013.json', orient='records')
gg1 = pd.read_json('gg2015.json', orient='records')

In [97]:
copy = gg.copy(deep=True)

In [98]:
def removePunc(x):
#     new_string = x.translate(str.maketrans('', '', string.punctuation))
    x = re.sub(r'[@#]\w+', '', x) #taking out hashtags and @ 
    x = re.sub(r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)', '', x) #taking out links 
    x = re.sub(r'[!?\.,\'\":()]+', '', x) #taking out punctuation i.e ? ! . ' : ( ) and " 
    x = re.sub(r'RT ', '', x) #taking out the initial "RT "
    return x.strip()
copy['text'] = copy['text'].str.lower()
copy['text'] = copy['text'].apply(lambda x: removePunc(str(x)))

In [99]:
test = list(copy['text'].values)
print(len(test))
s = list(set(test))
print(len(s))
df = pd.DataFrame(s, columns=['text'])

174643
121706


In [84]:
# def findTweets(contains, notcontains=[]):
#     data = df.loc[(df['text'].str.contains('|'.join(contains))) & (~df['text'].str.contains('|'.join(notcontains)))]
#     return data
# con = ['best', 'picture', 'musical', 'comedy', 'wins']
# noncon = ['actor', 'actress']
# t = findTweets(con, noncon)
# print(len(t))
# t.head(20)

In [93]:
df = df.loc[(df['text'].str.contains('best')) & 
            (df['text'].str.contains('picture')) & 
            (df['text'].str.contains('musical')) &
            (df['text'].str.contains('comedy')) &
            (~df['text'].str.contains('actor')) &
           (~df['text'].str.contains('actress')) &
           (df['text'].str.contains('wins'))]
print(len(df))
df.to_csv('awards.csv', index=False)
df.head(20)

72


,text
654,well deserved rt les miserables wins the gold...
3645,rt golden globes 2013 les miserables wins for...
5139,wins best motion picture comedy or musical
6197,rt “les miserables” wins best motion picture ...
6200,—&gt; rt les miserables wins best motion pict...
7334,wins best picture - drama and wins best pic...
8261,woot =] rt les miserables wins best picture m...
11314,rt wins best motion picture comedy/musical r...
11414,yes 🙌👌👍“ les miserables wins best motion pict...
11626,les miserables wins for best motion picture-co...


In [101]:
def findRows(x, contains, ncontains):
    s = "^"
    for word in contains:
        s += f'(?=.*{word})'
    for word in ncontains:
        s += f'(?!.*{word})'
    s += '.*$'
    if bool(re.match(s, x)):
        return True
    else:
        False
con = ['best', 'picture', 'musical', 'comedy', 'wins']
noncon = ['actor', 'actress']
df['Rows'] = df['text'].apply(lambda x: findRows(x, con, noncon))
print(len(df))
# df.head(20)

ccc = df[~df['Rows'].isnull()]
print(len(ccc))
hashNgrams(ccc)
# print(bool(re.match('^(?=.*best)(?=.*picture)(?=.*musical)(?=.*comedy)(?=.*wins)(?!.*actor)(?!.*actress).*$', p)))

121706
69


'les miserables'

In [57]:
def nGram(text):
    test = text
    firstsplit = test.split(' wins ', 1)
    if len(firstsplit) > 1:
        left = firstsplit[0].split(' ')
        left.reverse()
        new_list = []
        x = ""
        for i in left:
            if(i == ""):
                new_list.append(x) #HACKS
                continue
            x =  i + " " + x 
            new_list.append(x)
        new_list = [x.rstrip() for x in new_list]
    else:
        new_list = []
    return new_list

In [61]:
def hashNgrams(dff):
    FINAL_NGRAMS = []
    NGRAMS_DICT = {}
    for i in range(len(dff)):
        l = nGram(dff['text'].iloc[i])
        for j in l:
            if j in NGRAMS_DICT:
                NGRAMS_DICT[j] += 1
            else:
                NGRAMS_DICT[j] = 1
    cd = sorted(NGRAMS_DICT.items(),key=operator.itemgetter(1),reverse=True)
    return cd[0][0]
hashNgrams(df)

'les miserables'

In [ ]:
def findProperNouns(sen):
    tagged_sent = pos_tag(sen.split())
    propernouns = [word for word,pos in tagged_sent if pos == 'NNP']
    return propernouns
df['ProperNouns'] = df['text'].apply(lambda x: findProperNouns(x))
df.head(30)

In [ ]:
def removeStopwords(ppn):
    text = [word for word in ppn if word not in stopword]
    return text
df['ProperNounsFiltered'] = df['ProperNouns'].apply(lambda x: removeStopwords(x))
df.head(10)

In [ ]:
NGRAMS_DICT = {}
for i in range(len(df)):
    PN = df['ProperNounsFiltered'].iloc[i]
    for word in PN:
        if word in NGRAMS_DICT:
            NGRAMS_DICT[word] += 1
        else:
            NGRAMS_DICT[word] = 1
cd = sorted(NGRAMS_DICT.items(),key=operator.itemgetter(1),reverse=True)
cd